Import Libraries

In [1]:
import pandas as pd
import seaborn as sns

Read dataset and save it to a panda dataframe

In [2]:
df = pd.read_csv("D:/ML Projects/EDA_Ultra_Marathon/data/raw/TWO_CENTURIES_OF_UM_RACES.csv")
##change the dataset location here accordingly

C:\Users\siams\AppData\Local\Temp\ipykernel_427132\170188949.py:1: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("D:/ML Projects/EDA_Ultra_Marathon/data/raw/TWO_CENTURIES_OF_UM_RACES.csv")


See the first 10 datapoints

In [ ]:
df.head(10)

Shape and size of the frame

In [4]:
df.shape

(7461195, 13)

types of columns

In [ ]:
df.dtypes

Cleanup Data

Only want USA races, 50k or 50mile, 2020

Found 50km
50mi
50miles
for Event distance/length

In [ ]:
df[df['Event distance/length'] == '50miles']

Combine the different Event distance/length

In [ ]:
df[df['Event distance/length'].isin(['50km','50mi','50miles'])]

In [ ]:
df[df['Event distance/length'].isin(['50km','50mi','50miles']) & (df['Year of event'] == 2020)]

In [ ]:
df[df['Event name'].str.split('(').str.get(1).str.split(')').str.get(0) == 'USA']

combine all filters

In [10]:
filtered_df = df[df['Event distance/length'].isin(['50km','50mi','50miles']) & (df['Year of event'] == 2020) & (df['Event name'].str.split('(').str.get(1).str.split(')').str.get(0) == 'USA')]

In [ ]:
filtered_df.head(10)

Remove USA from event name

In [ ]:
filtered_df['Event name'] = filtered_df['Event name'].str.split('(').str.get(0)

In [ ]:
filtered_df.head(10)

Cleaner Athlete gender and age.

In [ ]:
filtered_df['athlete_age'] = 2020 - filtered_df['Athlete year of birth']
filtered_df['athlete_age'].head(10)

Remove h from athlete

In [ ]:
filtered_df['Athlete performance']=filtered_df['Athlete performance'].str.split(' ').str.get(0)

In [ ]:
filtered_df.head(10)

Drop cplumns: Athlete Club, Athlete Country, Athlete year of birth, athlete age of Category

In [17]:
filtered_df=filtered_df.drop(['Athlete club','Athlete country','Athlete year of birth','Athlete age category'], axis = 1)


In [ ]:
filtered_df.head(10)

Clean up null values

In [ ]:
filtered_df.isna().sum()

In [ ]:
filtered_df[filtered_df['athlete_age'].isna()==1]

In [21]:
filtered_df = filtered_df.dropna()

In [22]:
filtered_df.shape

(25857, 10)

Check for dupe

In [ ]:
filtered_df[filtered_df.duplicated()==True]

Reset Index

In [ ]:
filtered_df.reset_index(drop=True)

fix types

In [ ]:
filtered_df.dtypes

In [26]:
filtered_df['athlete_age'] = filtered_df['athlete_age'].astype(int)
filtered_df['Event name'] = filtered_df['Event name'].astype(str)
filtered_df['Athlete average speed'] = filtered_df['Athlete average speed'].astype(float)
filtered_df['Athlete gender'] = filtered_df['Athlete gender'].astype(str)


Rename Column

In [27]:
filtered_df = filtered_df.rename(columns = {'Year of event':'year',
                                            'Event dates':'race_day',
                                            'Event name':'race_name',
                                            'Event distance/length':'race_length',
                                            'Event number of finishers':'race_number_of_finishers',
                                            'Athlete performance':'athlete_performance',
                                            'Athlete gender':'athlete_gender',
                                            'Athlete ID':'athlete_id',
                                            'Athlete average speed':'athlete_average_speed'
                                            })

In [ ]:
filtered_df.head()

Reorder Cleanup

In [ ]:
filtered_df[filtered_df['race_name'] == 'Everglades 50 Mile Ultra Run ']

Charts and graph

In [ ]:
sns.histplot(filtered_df['race_length'])

In [ ]:
sns.histplot(filtered_df, x = 'race_length', hue = 'athlete_gender')

In [ ]:
sns.displot(filtered_df[filtered_df['race_length'] == '50mi']['athlete_average_speed'])

In [ ]:
sns.violinplot(filtered_df, x='race_length', y='athlete_average_speed', hue='athlete_gender', split = True, inner = 'quart', linewidth = 1)

In [ ]:
sns.lmplot(filtered_df, x = 'athlete_age',y="athlete_average_speed", hue = 'athlete_gender')

Question i want to find out from the data

Difference between speed for 50k,50mi male to female

In [ ]:
filtered_df.groupby(['race_length','athlete_gender'])['athlete_average_speed'].mean()

Age groups best in 50mi(20+races min)

In [ ]:
filtered_df.query('race_length == "50mi"').groupby('athlete_age')['athlete_average_speed'].agg(['mean','count']).sort_values('mean',ascending = False).query('count>19')

Seasons for the data -> Slower in summer than winter?

In [ ]:
filtered_df['race_month'] = filtered_df['race_day'].str.split('.').str.get(1).astype(int)
filtered_df.head()

In [38]:
filtered_df['race_season'] = filtered_df['race_month'].apply(lambda x: 'Winter' if x> 11 else 'Fall' if x>8 else 'Summer' if x>5 else 'Spring' if x>2 else 'Winter')

In [ ]:
filtered_df[filtered_df['race_month'] == 10]    

In [ ]:
filtered_df.groupby('race_season')['athlete_average_speed'].agg(['mean','count']).sort_values('mean', ascending = False)

50 miler only

In [ ]:
filtered_df.query('race_length == "50mi"').groupby('race_season')['athlete_average_speed'].agg(['mean','count']).sort_values('mean', ascending = False)